- https://medium.com/@vanillaxiangshuyang/self-supervised-learning-on-tabular-data-with-tabnet-544b3ec85cee
- https://colab.research.google.com/drive/1P8Obe07DP3VeOld08ThyT1HnChLip_LO#scrollTo=gvy9vUUNOP0W

- https://www.kaggle.com/code/sisharaneranjana/semi-supervised-pre-training-with-tabnet#%F0%9F%94%8FDescription-of-the-dataset-
- https://dacon.io/en/codeshare/3837

In [1]:
import pandas as pd
import numpy as np
import pickle

import warnings
warnings.simplefilter(action='ignore')

In [2]:
# 원본 데이터
with open("./dataset/creditcard.pkl","rb") as file:
    data = pickle.load(file)
    
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


# 데이터 전처리

In [3]:
df = data.copy()

# Time 을 일중 시간으로 변환
df.loc[:, "Time"] = df.loc[:, "Time"].apply(lambda x : x / 3600 % 24)

# Amount column 은 편차가 크므로 log-scale 로 변환
df['Amount'] = np.log(df.pop('Amount') + 0.001)

In [4]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Class,Amount
0,0.000000,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0,5.008105
1,0.000000,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,0,0.989913
2,0.000278,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,0,5.936641
3,0.000278,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0,4.816249
4,0.000556,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,0,4.248367


# train_test_split

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
features = df.values
labels = np.array(df.pop('Class'))

In [7]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.5, random_state=0, stratify=labels)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.5, random_state=0)

print("X train shape: ", X_train.shape)
print("X validation shape: ", X_val.shape)
print("X test shape: ", X_test.shape)
print("Y train shape: ", y_train.shape)
print("Y validation shape: ", y_val.shape)
print("Y test shape: ", y_test.shape)

X train shape:  (71201, 31)
X validation shape:  (71202, 31)
X test shape:  (142404, 31)
Y train shape:  (71201,)
Y validation shape:  (71202,)
Y test shape:  (142404,)


In [10]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
x_val= sc.transform(X_val)

# TabNetClassifier

https://github.com/dreamquark-ai/tabnet

In [13]:
# train on the whole dataset with labels
import torch
from pytorch_tabnet.tab_model import TabNetClassifier

supervised = TabNetClassifier(
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    scheduler_params={"step_size":10, # how to use learning rate scheduler
                      "gamma":0.9},
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    mask_type='sparsemax' # This will be overwritten if using pretrain model
)

In [14]:
from pytorch_tabnet.metrics import Metric
from sklearn.metrics import f1_score

class F1_Score(Metric):
    def __init__(self):
        self._name = "f1"
        self._maximize = True

    def __call__(self, y_true, y_score):
        score = f1_score(y_true, (y_score[:, 1]>0.5)*1)
        return score

In [15]:
supervised.fit(X_train, y_train,
               patience=5,
               eval_set=[(X_train, y_train), (x_val,y_val)],
               eval_metric=['logloss','f1']
)

epoch 0  | loss: 0.04237 | val_0_logloss: 0.01191 | val_0_f1: 0.03279 | val_1_logloss: 0.02337 | val_1_f1: 0.04286 |  0:00:08s
epoch 1  | loss: 0.00929 | val_0_logloss: 0.00751 | val_0_f1: 0.53801 | val_1_logloss: 0.03121 | val_1_f1: 0.27273 |  0:00:16s
epoch 2  | loss: 0.00557 | val_0_logloss: 0.00422 | val_0_f1: 0.76415 | val_1_logloss: 0.01935 | val_1_f1: 0.58883 |  0:00:25s
epoch 3  | loss: 0.00502 | val_0_logloss: 0.00488 | val_0_f1: 0.63212 | val_1_logloss: 0.02333 | val_1_f1: 0.49724 |  0:00:33s
epoch 4  | loss: 0.00531 | val_0_logloss: 0.0044  | val_0_f1: 0.74286 | val_1_logloss: 0.01511 | val_1_f1: 0.59113 |  0:00:41s
epoch 5  | loss: 0.00557 | val_0_logloss: 0.00428 | val_0_f1: 0.68783 | val_1_logloss: 0.01156 | val_1_f1: 0.46486 |  0:00:49s
epoch 6  | loss: 0.00495 | val_0_logloss: 0.00416 | val_0_f1: 0.79464 | val_1_logloss: 0.01137 | val_1_f1: 0.47887 |  0:00:57s
epoch 7  | loss: 0.00437 | val_0_logloss: 0.00299 | val_0_f1: 0.78261 | val_1_logloss: 0.02135 | val_1_f1: 0.18

In [24]:
predicted_test = supervised.predict_proba(X_test)
score = f1_score(y_test,predicted_test)
print(score)

ValueError: Classification metrics can't handle a mix of binary and continuous-multioutput targets

In [28]:
from pytorch_tabnet.pretraining import TabNetPretrainer

# TabNetPretrainer
unsupervised = TabNetPretrainer(
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    mask_type='entmax' # "sparsemax"
)

In [30]:
unsupervised.fit(X_train,
                 eval_set=[X_val],
                 pretraining_ratio=0.8,
)

epoch 0  | loss: 0.98345 | val_0_unsup_loss_numpy: 217.4924774169922|  0:00:06s
epoch 1  | loss: 0.34711 | val_0_unsup_loss_numpy: 266.1296691894531|  0:00:13s
epoch 2  | loss: 0.63993 | val_0_unsup_loss_numpy: 40.44477844238281|  0:00:20s
epoch 3  | loss: 0.27321 | val_0_unsup_loss_numpy: 289.6617431640625|  0:00:26s
epoch 4  | loss: 0.70554 | val_0_unsup_loss_numpy: 242.64297485351562|  0:00:33s
epoch 5  | loss: 0.75601 | val_0_unsup_loss_numpy: 222.50965881347656|  0:00:39s
epoch 6  | loss: -0.00715| val_0_unsup_loss_numpy: 589.055908203125|  0:00:46s
epoch 7  | loss: 0.48132 | val_0_unsup_loss_numpy: 167.38934326171875|  0:00:52s
epoch 8  | loss: 0.63309 | val_0_unsup_loss_numpy: 450.40167236328125|  0:00:59s
epoch 9  | loss: 0.79176 | val_0_unsup_loss_numpy: 107.77104949951172|  0:01:06s
epoch 10 | loss: 0.78177 | val_0_unsup_loss_numpy: 46.169281005859375|  0:01:12s
epoch 11 | loss: 0.05704 | val_0_unsup_loss_numpy: 731.3218383789062|  0:01:19s
epoch 12 | loss: 0.69741 | val_0_un

# Pre-trained 된 모델로 test

In [31]:
clf = TabNetClassifier(
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    scheduler_params={"step_size":10, # how to use learning rate scheduler
                      "gamma":0.9},
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    mask_type='sparsemax' # This will be overwritten if using pretrain model
)

In [32]:
clf.fit(X_train,y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],
    eval_name=['train', 'valid'],
    eval_metric=['logloss','f1'],
    from_unsupervised=unsupervised
)

epoch 0  | loss: 0.0316  | train_logloss: 0.00926 | train_f1: 0.25676 | valid_logloss: 0.01494 | valid_f1: 0.01493 |  0:00:07s
epoch 1  | loss: 0.00853 | train_logloss: 0.00737 | train_f1: 0.50538 | valid_logloss: 0.01078 | valid_f1: 0.27381 |  0:00:16s
epoch 2  | loss: 0.00638 | train_logloss: 0.0048  | train_f1: 0.65766 | valid_logloss: 0.00938 | valid_f1: 0.62264 |  0:00:24s
epoch 3  | loss: 0.00485 | train_logloss: 0.00377 | train_f1: 0.76667 | valid_logloss: 0.00722 | valid_f1: 0.69565 |  0:00:31s
epoch 4  | loss: 0.00409 | train_logloss: 0.00303 | train_f1: 0.8087  | valid_logloss: 0.00764 | valid_f1: 0.6749  |  0:00:38s
epoch 5  | loss: 0.00381 | train_logloss: 0.00328 | train_f1: 0.82819 | valid_logloss: 0.00808 | valid_f1: 0.69492 |  0:00:46s
epoch 6  | loss: 0.00293 | train_logloss: 0.00378 | train_f1: 0.8658  | valid_logloss: 0.00755 | valid_f1: 0.66667 |  0:00:53s
epoch 7  | loss: 0.00248 | train_logloss: 0.00204 | train_f1: 0.89831 | valid_logloss: 0.00671 | valid_f1: 0.68

In [34]:
predicted_test=clf_partial.predict_proba(x_test)[:,1]
score=roc_auc_score(y_test,predicted_test)
print(score)

NameError: name 'clf_partial' is not defined